## Custom YOLO11 Object Detection Model — Google Colab Version
- Dataset: Downloaded from Roboflow via API (≥100 images)
- Model: YOLO11n (nano)
- Inference: Batch process 20 test images → output saved to folder
- No manual image-by-image handling — fully automated!

In [ ]:
# Install required packages
!pip install ultralytics roboflow

In [ ]:
# Import libraries
from roboflow import Roboflow
from ultralytics import YOLO
import os
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# load roboflow api key from Colab Secrets Manager
from google.colab import userdata
SECRET_KEY = userdata.get('ROBOFLOW_API_KEY')

if SECRET_KEY is None:
    raise ValueError("Roboflow API key not found in Colab Secrets Manager. Please add it with the name 'ROBOFLOW_API_KEY'.")
else:
    print("Roboflow API key loaded")

Roboflow API key loaded


In [ ]:
# STEP 1: DOWNLOAD DATASET FROM ROBOFLOW USING API KEY

RF_API_KEY = SECRET_KEY  # Use the variable holding the API key
PROJECT_NAME = "basketball-players-fy4c2-e42zi" # project name from roboflow web
VERSION = 1                           # dataset version number from roboflow web

rf = Roboflow(api_key=RF_API_KEY) # Initialized Roboflow with the API key directly

print("Downloading dataset from Roboflow...")
project = rf.workspace().project(PROJECT_NAME) # Access the workspace without arguments
dataset = project.version(VERSION).download("yolo11")  # Downloads as YOLO11 format

dataset_path = f"/content/drive/MyDrive/Datasets/{PROJECT_NAME}-{VERSION}"  # Path to downloaded dataset - using project name and version for clarity
print(f"Dataset saved at: {dataset_path}")

loading Roboflow workspace...
loading Roboflow project...

Version export complete for yolo11 format



Extracting Dataset Version Zip to Basketball-Players-1 in yolo11:: 100%|██████████| 352/352 [00:00<00:00, 4076.04it/s]

Dataset saved at: /content/drive/MyDrive/Datasets/basketball-players-fy4c2-e42zi-1


In [ ]:
# STEP 2: TRAIN CUSTOM YOLO11 MODEL

import torch # Import torch

# Define the dataset path again to ensure it's correct in this cell
PROJECT_NAME = "basketball-players-fy4c2-e42zi"
VERSION = 1
dataset_path = "/content/Basketball-Players-1"

model = YOLO("yolo11n.pt")  # Nano model — fastest for training/inference

print("\nStarting training...")
results = model.train(
    data=f"{dataset_path}/data.yaml",  # Path to data.yaml
    epochs=50,
    imgsz=640,
    batch=16,
    name="custom_yolo11n",
    device=0 if torch.cuda.is_available() else 'cpu'  # Use GPU if available
)

print("Training completed!")


Starting training...
Ultralytics 8.3.205 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Basketball-Players-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=custom_yolo11n3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=

In [9]:
# STEP 3: BATCH INFERENCE ON TEST IMAGES
# Get test images from the dataset's test folder
test_images_folder = f"{dataset_path}/test/images"


In [10]:
# Create output folder
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

In [11]:
# List all test images (jpg, png, jpeg)
test_images = [f for f in os.listdir(test_images_folder)
               if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

print(f"\nFound {len(test_images)} test images. Running inference...")


Found 24 test images. Running inference...


In [16]:
# Load trained model
model = YOLO("/content/runs/detect/custom_yolo11n3/weights/best.pt")

# Perform inference on all test images
results = model(test_images_folder)


image 1/24 /content/Basketball-Players-1/test/images/Iowa_Nebraska-2023-03-06-college-men_mp4-106_jpg.rf.58f791f9075215a56c053df5a4da41e3.jpg: 384x640 1 Hoop, 11 Players, 2 Refs, 49.7ms
image 2/24 /content/Basketball-Players-1/test/images/Iowa_Nebraska-2023-03-06-college-men_mp4-124_jpg.rf.9ab4d3c4f1655d8d518d67365b76a7d9.jpg: 384x640 1 Hoop, 12 Players, 3 Refs, 10.1ms
image 3/24 /content/Basketball-Players-1/test/images/Iowa_Nebraska-2023-03-06-college-men_mp4-97_jpg.rf.22bbfcd6b03d2e0935b76558d9917a75.jpg: 384x640 1 Hoop, 13 Players, 2 Refs, 8.0ms
image 4/24 /content/Basketball-Players-1/test/images/youtube-0_jpg.rf.b22c9e3340726a8c836515dfc12f14b8.jpg: 384x640 13 Players, 2 Refs, 2 Team Pointss, 2 Time Remainings, 8.5ms
image 5/24 /content/Basketball-Players-1/test/images/youtube-114_jpg.rf.ff9b1f0ec33de13a86c7b9f583741d4d.jpg: 384x640 4 Players, 3 Team Pointss, 2 Time Remainings, 8.2ms
image 6/24 /content/Basketball-Players-1/test/images/youtube-128_jpg.rf.030a921e404cae04553ed5c1

In [17]:
# Loop through all test images
for i, result in enumerate(results[:20]):  # Process first 20 images and iterate through results
    img_file = test_images[i] # Get the original image filename
    output_path = os.path.join(output_folder, img_file)

    # Save result image with bounding boxes
    result.save(filename=output_path)

    print(f"Saved: {output_path}")

print(f"\nProcessed and saved results for the first 20 test images to '{output_folder}' folder.")

Saved: output/youtube-0_jpg.rf.b22c9e3340726a8c836515dfc12f14b8.jpg
Saved: output/youtube-65_jpg.rf.78dd2ac35e1ff02d5cf3232ac7182ce1.jpg
Saved: output/youtube-13_jpg.rf.a777dbe3471edec0664f442edd7dd12e.jpg
Saved: output/youtube-153_jpg.rf.3932aa7247ad95a56420c82dd29f2bcf.jpg
Saved: output/youtube-29_jpg.rf.00b20fec582aeb507e191cdf09dcf540.jpg
Saved: output/youtube-18_jpg.rf.081f0a633214eda8c6ddae25bc423dc9.jpg
Saved: output/youtube-201_jpg.rf.6461310cae11facb5e80b1456f5eb21e.jpg
Saved: output/youtube-114_jpg.rf.ff9b1f0ec33de13a86c7b9f583741d4d.jpg
Saved: output/youtube-9_jpg.rf.f1960ece0187489ee1d5d6b915f84ab7.jpg
Saved: output/youtube-230_jpg.rf.d27e969ce96580f5bac466645343d68a.jpg
Saved: output/youtube-151_jpg.rf.86922c0d0bb231656353add33233c22a.jpg
Saved: output/youtube-2_jpg.rf.4f126ce80ed6e9775f3fcc08d01b2a14.jpg
Saved: output/youtube-23_jpg.rf.0456f7fe54cb39c07776fe9a80a40ae4.jpg
Saved: output/youtube-15_jpg.rf.deab89215f9fb6fcadad7234b46c5d5d.jpg
Saved: output/Iowa_Nebraska-2023

In [18]:
# Download output folder as ZIP
from google.colab import files
!zip -r output.zip output/
files.download('output.zip')

  adding: output/ (stored 0%)
  adding: output/youtube-0_jpg.rf.b22c9e3340726a8c836515dfc12f14b8.jpg (deflated 3%)
  adding: output/youtube-65_jpg.rf.78dd2ac35e1ff02d5cf3232ac7182ce1.jpg (deflated 3%)
  adding: output/youtube-13_jpg.rf.a777dbe3471edec0664f442edd7dd12e.jpg (deflated 3%)
  adding: output/youtube-153_jpg.rf.3932aa7247ad95a56420c82dd29f2bcf.jpg (deflated 3%)
  adding: output/youtube-29_jpg.rf.00b20fec582aeb507e191cdf09dcf540.jpg (deflated 5%)
  adding: output/youtube-18_jpg.rf.081f0a633214eda8c6ddae25bc423dc9.jpg (deflated 3%)
  adding: output/youtube-201_jpg.rf.6461310cae11facb5e80b1456f5eb21e.jpg (deflated 4%)
  adding: output/youtube-114_jpg.rf.ff9b1f0ec33de13a86c7b9f583741d4d.jpg (deflated 3%)
  adding: output/youtube-9_jpg.rf.f1960ece0187489ee1d5d6b915f84ab7.jpg (deflated 10%)
  adding: output/youtube-230_jpg.rf.d27e969ce96580f5bac466645343d68a.jpg (deflated 3%)
  adding: output/youtube-151_jpg.rf.86922c0d0bb231656353add33233c22a.jpg (deflated 3%)
  adding: output/you

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Key files to look for in `/content/runs/detect/custom_yolo11n3`:**

*   **`results.csv`**: Contains detailed training metrics for each epoch (loss, precision, recall, mAP, etc.).
*   **`confusion_matrix.png`**: Visualizes the performance of the model in terms of true positives, true negatives, false positives, and false negatives for each class.
*   **`F1_curve.png`, `P_curve.png`, `R_curve.png`**: Plots of F1 score, Precision, and Recall against confidence thresholds.
*   **`labels.jpg`**: Visualizations of the training and validation labels.
*   **`train_batch*.jpg`, `val_batch*.jpg`**: Visualizations of training and validation batches with augmentation applied and bounding boxes.
*   **`weights/`**: This folder contains the trained model weights (`best.pt` and `last.pt`).